# Rainfall Analysis and Prediction

---

**Author:** [rodoart](https://github.com/rodoart/)<br>
**Date created:** 2023-01-29 <br>
**Last modified:** 2023-01-30<br>
**Description:** Downloading and formating the data set from  [Mexican Meteorological Service](https://smn.conagua.gob.mx/es/climatologia/informacion-climatologica/informacion-estadistica-climatologica) with info about rainfall in Mexico. This procedure is part of `rainfall.data.make_dataset` module. This notebook has been kept for illustrative purposes only.


# 1. Data download and preparation


## Path config

If you want the files to be copied to another folder within the same machine you are working on, by a source path other than remote.



In [17]:
PROJECT_SLUG = 'rainfall'
NAME = 'data_preparation'
NUMBER = '1.0'

NOTEBOOK_NAME = f'{NUMBER}_{PROJECT_SLUG}-{NAME}.ipynb'


# LOCAL
LOCAL_PATH = '..'

In [18]:
import sys

# It depends on where the library that comes with this package is stored.
sys.path.append(LOCAL_PATH)

In [3]:
from rainfall.utils.paths import make_dir_function

In [15]:
local_dir = make_dir_function(workspace=LOCAL_PATH)

## Data download, cleaning and preparation

The data comes from the [Mexican Meteorological Service](https://smn.conagua.gob.mx/es/climatologia/informacion-climatologica/informacion-estadistica-climatologica). They are available as a Google Earth `.kmz` file, which shows the more than 1000 stations in the republic and links to download it.

![Image of weather stations in Google Earth Pro.](https://i.ibb.co/5WLZNSQ/republica.jpg)



### Data of stations

Download the `kmz` file from the url.

In [19]:
from os import makedirs
makedirs(local_dir('tmp'), exist_ok=True)
makedirs(local_dir('data'), exist_ok=True)

In [78]:
import requests

file_name = 'EstacionesClimatologicas.kmz'
kmz_url = f'https://smn.conagua.gob.mx/tools/RESOURCES/estacion/{file_name}'
kmz_path = local_dir('tmp', 'weather_stations.kmz')


response = requests.get(kmz_url, verify=False)
open(kmz_path, "wb").write(response.content)

c:\Users\rodoa\.conda\envs\rainfall\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smn.conagua.gob.mx'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


542532

Files of type `kmz` are a zipped form of `kml` files with images and other files.  `kml` that follow an `xml` structure and contain geographic information and are used in GIS (Geographic Information Systems) programs. We just have to unzip them.

In [41]:
import zipfile

kml_name = 'weather_stations'
unzip_path = local_dir('tmp', kml_name)

with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

Copying only the `.kml` file to the raw data folder.

In [43]:
kml_origin

WindowsPath('C:/files/proyects/rainfall/tmp/weather_stations/doc.kml')

In [44]:
from shutil import copy

kml_origin = unzip_path.joinpath('doc.kml')
kml_path = local_dir('data', 'raw', f'{kml_name}.kml')


copy(kml_origin, kml_path)

WindowsPath('C:/files/proyects/rainfall/data/raw/weather_stations.kml')

This is a peek of the file:

In [49]:
kml_file = open(kml_path, "r", encoding='utf-8')
lines = kml_file.readlines() 


for k in range(87,103):
  print(lines[k][:-1])
  

			<Placemark>
				<name>1011</name>
				<description><![CDATA[<h3>MALPASO - OPERANDO</h3><font color='red'><p><b>Estado : </b>AGUASCALIENTES</p><p><b>Municipio : </b>CALVILLO</p><p><b>Organismo : </b>CONAGUA-DGE</p><p><b>Cuenca : </b>RIO JUCHIPILA</p><p><a href=https://smn.conagua.gob.mx/tools/RESOURCES/Diarios/1011.txt>Climatología diaria</a></p><p><a href=  https://smn.conagua.gob.mx/tools/RESOURCES/Estadistica/1011.pdf href=  https://smn.conagua.gob.mx/tools/RESOURCES/Normales5110/NORMAL01011.TXT>Normales 1951-2010</a></p><p><a href=  https://smn.conagua.gob.mx/tools/RESOURCES/Normales7100/NORMAL01011.TXT >Normales 1971-2000</a></p><p><a href=  https://smn.conagua.gob.mx/tools/RESOURCES/Normales8110/NORMAL01011.TXT >Normales 1981-2010</a></p><p><a href=https://smn.conagua.gob.mx/tools/RESOURCES/Max-Extr/00001/00001011.TXT>Valores Extremos</a></p><p><a href=https://smn.conagua.gob.mx/tools/RESOURCES/Mensuales/ags/00001011.TXT>Valores Mensuales</a></p>]]></description>
				<styleUrl>

The information we are looking for is inside the `<Placemark>` tag which contains the number under the `<name>` tag, the coordinates as `<coordinates>` and more relevant data in `<description>`.

Regular expressions are used to extract this information.

In [50]:
import regex as re

In [51]:
# Regular expression to extract description tag.
r_stations = '^\s*<description><!\[CDATA\[<h3>' +\
             "(.*) - ([A-Z]{8,10})</h3><font color='red'><p><b>Estado : </b>"+ \
             '(.*)</p><p><b>Municipio : </b>' +\
             '(.*)</p><p><b>Organismo : </b>' +\
             '(.*)</p><p><b>Cuenca : </b>'+\
             '(.*)</p><p><a href=https://smn.conagua.gob.mx/tools/RESOURCES/Diarios/'+\
             '(.*).txt>Climatología diaria</a></p>.*</a></p>\]\]></description>$'

rs = re.compile(r_stations) 

# Regular expression to extract coordinates tag.
r_coordinates = '^\s*<coordinates>(.*),(.*),(.*)</coordinates>$'
rc = re.compile(r_coordinates) 

Search with both regular expresions line by line:

In [52]:
rows = []
for k, line in enumerate(lines):
  if rs.match(line):
    rows.append(rs.search(line).groups() + rc.search(lines[k+3]).groups())

rows[0]

('CAÑADA HONDA',
 'OPERANDO',
 'AGUASCALIENTES',
 'AGUASCALIENTES',
 'CONAGUA-DGE',
 'RIO VERDE GRANDE',
 '1004',
 '-102.1989',
 '22.0008',
 '1925')

Now a data frame is defined with the information obtained.

In [53]:
import pandas as pd

In [54]:
cols = ['name', 'status', 'state', 'municipality', 'organization', 'basin', 'number', 'longitude', 'latitude', 'altitude']
df_stations = pd.DataFrame(rows, columns=cols)
df_stations.head(5)

,name,status,state,municipality,organization,basin,number,longitude,latitude,altitude
0,CAÑADA HONDA,OPERANDO,AGUASCALIENTES,AGUASCALIENTES,CONAGUA-DGE,RIO VERDE GRANDE,1004,-102.1989,22.0008,1925
1,PRESA EL NIAGARA,OPERANDO,AGUASCALIENTES,AGUASCALIENTES,CONAGUA-DGE,RIO VERDE GRANDE,1005,-102.3717,21.7806,1844
2,PUERTO DE LA CONCEPCION,OPERANDO,AGUASCALIENTES,TEPEZALA,CONAGUA-DGE,RIO VERDE GRANDE,1008,-102.135,22.2028,2322.8
3,LA TINAJA,OPERANDO,AGUASCALIENTES,SAN JOSE DE GRACIA,CONAGUA-DGE,RIO VERDE GRANDE,1010,-102.5542,22.1644,2525.7
4,MALPASO,OPERANDO,AGUASCALIENTES,CALVILLO,CONAGUA-DGE,RIO JUCHIPILA,1011,-102.6639,21.86,1730.2


Correcting formatting and type issues.

In [55]:
# Change names and types
df_stations = df_stations.astype({
    'number':'int64','status':'category', 'longitude':'float64', 
    'latitude':'float64', 'altitude':'float64'
    })
df_stations['status'] = df_stations['status'].cat.rename_categories({'OPERANDO':'working', 'SUSPENDIDA':'stopped'})

In [56]:
df_stations = df_stations.set_index('number')
df_stations.sort_index()

df_stations.head()

,name,status,state,municipality,organization,basin,longitude,latitude,altitude
number,,,,,,,,,
1004,CAÑADA HONDA,working,AGUASCALIENTES,AGUASCALIENTES,CONAGUA-DGE,RIO VERDE GRANDE,-102.1989,22.0008,1925.0
1005,PRESA EL NIAGARA,working,AGUASCALIENTES,AGUASCALIENTES,CONAGUA-DGE,RIO VERDE GRANDE,-102.3717,21.7806,1844.0
1008,PUERTO DE LA CONCEPCION,working,AGUASCALIENTES,TEPEZALA,CONAGUA-DGE,RIO VERDE GRANDE,-102.1350,22.2028,2322.8
1010,LA TINAJA,working,AGUASCALIENTES,SAN JOSE DE GRACIA,CONAGUA-DGE,RIO VERDE GRANDE,-102.5542,22.1644,2525.7
1011,MALPASO,working,AGUASCALIENTES,CALVILLO,CONAGUA-DGE,RIO JUCHIPILA,-102.6639,21.8600,1730.2


Exporting the data frame.

In [60]:
makedirs(local_dir('data','processed'), exist_ok=True)

In [61]:
df_stations = df_stations.reset_index()
df_stations.to_feather(
    local_dir('data','processed', 'df_stations.ftr'))

### Download files of a specific region

Now, it's time to use the data from the dataset to download the information for a specific region.

Each station has an online `.txt` document that contains information about the station number, name, state, geographic location, etc. and a table with daily weather data for precipitation, minimum and maximum temperature, and humidity. Here is an example:

```
CNA-SMN-CG-GMC-SMAA-CLIMATOLOGIA
BASE DE DATOS CLIMATOLOGICA
DATOS DISPONIBLES EN LA BASE DE DATOS A MARZO 2020;CON LA INFORMACION SUMINISTRADA POR LAS OFICINAS REGIONALES
 
ESTACION  : 10073
NOMBRE    : SANTA BARBARA (SMN)
ESTADO    : DURANGO
MUNICIPIO : DURANGO
SITUACIÓN : SUSPENDIDA
ORGANISMO : CONAGUA-SMN
CVE-OMM   : Nulo
LATITUD   : 023.783°
LONGITUD  : -104.900°
ALTITUD   : 2,316 msnm
 
EMISION   : 06/04/2020 
 
           PRECIP  EVAP   TMAX   TMIN
  FECHA     (MM)   (MM)   (°C)   (°C)
03/10/1961  40     3.1    28     12 
04/10/1961  0      3.1    21     10 
05/10/1961  0      0.5    26     18 
06/10/1961  0      3.1    24     6.5 
07/10/1961  7      0.3    22     6 

...
```

It is necessary to extract the information from the stations in order to have the categorized data. Fortunately, all the data is on pages whose url is:

<https://smn.conagua.gob.mx/tools/RESOURCES/Diarios/{station_number}.txt>

where `station number` is a number between 10,000 and 33,000.

First, we obtain the station numbers of the required region. In this case we use the city of Durango in the homonymous state.

In [62]:
df_stations = pd.read_feather(local_dir('data','processed', 'df_stations.ftr'))
df_stations = df_stations.set_index('number')

In [63]:
selected_state = 'durango'
selected_municipality = 'durango'

station_numbers = df_stations.query(
    f"state == '{selected_state.upper()}' and "+
    f"municipality == '{selected_municipality.upper()}'"
).index.to_list()

station_numbers[:5]

[10017, 10023, 10024, 10040, 10048]

#### Asynchronous data download

We will download files from all listed stations in parallel.

We use the `aiohttp` library instead of `requests`, because many queries must be done quickly.

In [64]:
!pip install aiohttp
!pip install aiofiles

In [65]:
import aiohttp
import asyncio
import aiofiles

# Only for notebooks
import nest_asyncio
nest_asyncio.apply()

# Only for Windows
from sys import platform
if  platform.startswith('win'):
    policy = asyncio.WindowsSelectorEventLoopPolicy()
    asyncio.set_event_loop_policy(policy)

Using mozilla as header a create a list with all of the urls.

In [66]:
mozilla = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0'
}

urls = [f'https://smn.conagua.gob.mx/tools/RESOURCES/Diarios/{number}.txt' 
        for number in station_numbers
       ]

In [67]:
download_dir = local_dir('data','raw', selected_state, selected_municipality)
makedirs(download_dir, exist_ok=True)

In [68]:
limit = 3
http_ok = [200]


async def scrape(url_list):
    tasks = list()
    sem = asyncio.Semaphore(limit)

    async with aiohttp.ClientSession(trust_env=True, headers=mozilla) as session:
        for url in url_list:
            task = asyncio.ensure_future(scrape_bounded(url, sem, session))
            tasks.append(task)

        result = await asyncio.gather(*tasks)

    return result

async def scrape_bounded(url, sem, session):
    async with sem:
        new_line = await scrape_one(url, session)
        return new_line


async def scrape_one(url, session):
  file_name = url.rsplit('/', 1)[-1]

  try:
      async with session.get(url, verify_ssl=False) as response:
          content = await response.text(encoding='cp1252')

          async with aiofiles.open(download_dir.joinpath(file_name),
                                   mode = 'w') as f:
            await f.write(content)

  except aiohttp.client_exceptions.ClientConnectorError:
      print('Scraping  failed due to the connection problem', url)
      return False

  if response.status not in http_ok:
      print('Scraping failed due to the return code ', url, response.status)
      return False        

  return file_name


loop = asyncio.get_event_loop()
lines = loop.run_until_complete(scrape(urls))

### Create a data frame with all the data in the region

In [69]:
from os import listdir

In [70]:
file_names = [file for file in listdir(download_dir) if file.endswith('.txt')]

In [71]:
# Store results.
df_all = None

# Loading files.
for file_name in file_names:
  file_path = download_dir.joinpath(file_name)

  try:
    df_rainfall = pd.read_csv(file_path, skiprows=19, 
                            skipfooter=1,
                            delim_whitespace=True,
                            na_values = ['Nulo'], 
                            encoding='cp1252',
                            engine='python',
                            names=['date', 'rainfall','evaporation', 'max_t', 'min_t'])
  
    df_rainfall['date']=pd.to_datetime(df_rainfall['date'], format='%d/%m/%Y')

    if df_all is not None:  
      df_all = pd.concat([df_all, df_rainfall])
    else:
      df_all = df_rainfall.copy()
  except Exception as e:
    if not e.__class__== UnicodeDecodeError:
      print('\nSomething bad has happened.')  

In [72]:
len(df_all)

266971

As there are repeated data, the average of each day is used. It is ignored if the data is null, in this way some gaps are filled.

In [73]:
df_all = df_all.groupby('date').median().reset_index()
len(df_all)

28124

In [74]:
df_all.isna().sum()

date            0
rainfall        1
evaporation    51
max_t           0
min_t           0
dtype: int64

Remaining null values are removed by simple interpolation.

In [75]:
df_all = df_all.set_index('date')

In [76]:
df_all = df_all.interpolate(method='time')

In [77]:
df_all = df_all.reset_index()
df_all.to_feather(local_dir('data', 'processed', f'df_{selected_state}_{selected_municipality}_medians.ftr'))